# Tags

In [8]:
!gcloud config set project etsy-bigquery-adhoc-prod


If you have a compatible Python interpreter installed, you can use it by setting
the CLOUDSDK_PYTHON environment variable to point to it.

Updated property [core/project].


In [9]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [10]:
import numpy as np
from tqdm import tqdm

In [11]:
%%bigquery tags_df
select lower(listingWeb_tags) as listingWeb_tags, lower(verticaListings_tags) as verticaListings_tags
from `etsy-ml-systems-prod.feature_bank_v2.listing_feature_bank_2024-02-08`
where EDIT_DISTANCE(lower(listingWeb_tags), lower(verticaListings_tags)) > 1
and listingWeb_tags != ''
and verticaListings_tags != ''

Query is running:   0%|          |

Downloading:   0%|          |

In [12]:
tags_df.head()

,listingWeb_tags,verticaListings_tags
0,baking gifts.baker shirt.bakery shirt.pastry c...,.baking gifts.baker shirt.bakery shirt.pastry ...
1,dior sports sweater.mixed knit sweater.vintage...,clothing.women.dior sports sweater.mixed knit ...
2,booksshop.teamvintageusa.vintage retro old.mid...,supplies.booksshop.teamvintageusa.vintage retr...
3,brown vintage glove.button trim gloves.embelli...,accessories.gloves.brown vintage glove.button ...
4,goodluck mug.colleague mug.goodbye mug.colleag...,housewares.goodluck mug.colleague mug.goodbye ...


In [13]:
tags_df.shape

(22270244, 2)

In [14]:
lw_count = 0
vl_count = 0
for i in tqdm(range(tags_df.shape[0])):
    lw_tags = tags_df.listingWeb_tags[i].split('.')
    vl_tags = tags_df.verticaListings_tags[i].split('.')
    
    if lw_tags[0] == '':
        lw_tags = lw_tags[1:]
    if vl_tags[0] == '':
        vl_tags = vl_tags[1:]
        
    if len(lw_tags) > len(vl_tags):
        diff_tags = set(lw_tags) - set(vl_tags)
        diff_tags = list(diff_tags)
        if len(diff_tags) > 0:
            lw_count += 1
    else:
        diff_tags = set(vl_tags) - set(lw_tags)
        diff_tags = list(diff_tags)
        if len(diff_tags) > 0:
            vl_count += 1

100%|████████████████████████████████████████████████████████████████| 22270244/22270244 [08:41<00:00, 42704.55it/s]


In [15]:
lw_count

9293098

In [16]:
vl_count

11514323